# Final Project - Part 1
## Problem Statement
My goal with this analysis is to determine if the Vaccine Adverse Event Reporting System (VAERS) data can help predict a patient's outcome given their adverse symptoms or other relevant information from taking a given vaccine.  This will be a binary classification problem where the model's made will determine if the patient DIED (y=1) or LIVED (y=0) from their adverse event report.  
I am solving this problem in order to better understand the severity of adverse reactions people can have from certain vaccines.

# 1. Exploratory Data Analysis (EDA)

**Dataset**: “VAERS Data Sets”


**Dataset Source**: https://vaers.hhs.gov/data/datasets.html



**Key attributes/dimensions of the data**: 
There are 3 main CSV files associated with this dataset:
1)	*VAERSDATA.csv 
2)	*VAERSSYMPTOMS.csv
3)	*VAERSVAX.csv

These CSV files contain numbers, dates, and other unique values that can be mapped to their Vaccine Adverse Event Reporting System (VAERS) identification number for each person among the spreadsheets.  The VAERS was made by the Food and Drug Administration (FDA) and the Centers for Disease Control and Prevention (CDC) to collect data on adverse reactions that could be associated with vaccines in general.  

I pulled this data on 9/6/2024, which stated the data contains VAERS reports processed as of **08/30/2024**.

I filtered to only include those vaccines related to COVID19.  This dataset includes false positives for adverse reactions given a vaccine due to the encouragement towards doctors and other vaccine providers to report any adverse event if seen regardless of if they can prove the vaccine was the cause.  Therefore, the dataset contains a combination of coincidental events and those truly caused by the vaccine.  

The dataset I downloaded I have shared here on my GDRIVE for people to reproduce my work: 

https://drive.google.com/file/d/13ft2QazL4OugjftllcwjPXLUVMB8-2Xi/view?usp=drive_link



**Import Libraries**

In [2]:
import pandas as pd
import altair as alt
import os
import numpy as np
from statsmodels.stats.power import GofChisquarePower
from scipy.stats import binomtest
import warnings
warnings.filterwarnings("ignore")

**Import Data**

In [3]:
MAIN_DIRS = "AllVAERSDataCSVS"
ALL_CSVS = os.listdir(MAIN_DIRS)
VAERSDATA_CSVS = [w for w in ALL_CSVS if w.find("DATA")!=-1 and w.lower().find("nondomestic")==-1]
VAERSSYMP_CSVS = [w for w in ALL_CSVS if w.find("SYMPTOMS")!=-1 and w.lower().find("nondomestic")==-1]
VAERSVAX_CSVS = [w for w in ALL_CSVS if w.find("VAX")!=-1 and w.lower().find("nondomestic")==-1]

In [4]:
for i in range(len(VAERSDATA_CSVS)):
    if i==0:
        df_VAERSDATA = pd.read_csv(os.path.join(MAIN_DIRS,VAERSDATA_CSVS[i]),encoding='ISO-8859-1')
    else:
        df_i = pd.read_csv(os.path.join(MAIN_DIRS,VAERSDATA_CSVS[i]),encoding='ISO-8859-1')
        df_VAERSDATA = pd.concat([df_VAERSDATA,df_i],axis=0)
        df_VAERSDATA = df_VAERSDATA.reset_index().drop("index",axis=1)

In [5]:
df_VAERSDATA

,VAERS_ID,RECVDATE,STATE,AGE_YRS,CAGE_YR,CAGE_MO,SEX,RPT_DATE,SYMPTOM_TEXT,DIED,...,CUR_ILL,HISTORY,PRIOR_VAX,SPLTTYPE,FORM_VERS,TODAYS_DATE,BIRTH_DEFECT,OFC_VISIT,ER_ED_VISIT,ALLERGIES
0,25001,07/02/1990,WI,0.20,NaN,NaN,F,NaN,Loud intense cry with screaming for 1 1/2 hrs....,NaN,...,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN
1,25003,07/02/1990,TX,0.80,NaN,NaN,M,NaN,"Hypotonic, Hyporesponsive episode, Infant died...",Y,...,NaN,NaN,~ ()~~~In patient,NaN,1,NaN,NaN,NaN,NaN,NaN
2,25004,07/02/1990,NY,0.90,NaN,NaN,M,NaN,"Pt developed chills for approx. 1 hr, felt ach...",NaN,...,NaN,NaN,~ ()~~~In patient,890269201,1,NaN,NaN,NaN,NaN,NaN
3,25005,07/02/1990,OK,NaN,NaN,NaN,U,NaN,7 patients within 2 weeks have reported joint ...,NaN,...,NaN,NaN,~ ()~~~In patient,890277901,1,NaN,NaN,NaN,NaN,NaN
4,25006,07/02/1990,OH,16.00,16.0,NaN,F,NaN,16 yr old female feeling faint & then had seiz...,NaN,...,NaN,no hx of local or systemic rxns,~ ()~~~In patient,890278001,1,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1855857,2787668,08/30/2024,IN,37.00,37.0,NaN,F,NaN,"Skin reactions, allergies, muscle weakness, bo...",NaN,...,NaN,"Arthrisis, mild- moderate headaches, endometri...",NaN,NaN,2,08/30/2024,Y,Y,NaN,Tramadol
1855858,2787674,08/30/2024,GA,0.25,0.0,0.3,U,NaN,No additional AE/No PQC.; HCP reported that RE...,NaN,...,NaN,NaN,NaN,US0095075132408USA010106,2,08/30/2024,NaN,NaN,NaN,NaN
1855859,2787675,08/30/2024,NY,NaN,NaN,NaN,F,NaN,Shingles outbreak after vaccination; This non-...,NaN,...,NaN,NaN,NaN,USGSKUS2024105377,2,08/30/2024,NaN,NaN,NaN,NaN
1855860,2787676,08/30/2024,KS,0.17,0.0,0.2,M,NaN,nurse stated that she accidentally gave ACTHIB...,NaN,...,NaN,NaN,NaN,USSA2024SA252142,2,08/30/2024,NaN,NaN,NaN,NaN


In [6]:
for i in range(len(VAERSSYMP_CSVS)):
    if i==0:
        df_VAERSSYMPTOMS = pd.read_csv(os.path.join(MAIN_DIRS,VAERSSYMP_CSVS[i]),encoding='ISO-8859-1')
    else:
        df_i = pd.read_csv(os.path.join(MAIN_DIRS,VAERSSYMP_CSVS[i]),encoding='ISO-8859-1')
        df_VAERSSYMPTOMS = pd.concat([df_VAERSSYMPTOMS,df_i],axis=0)
        df_VAERSSYMPTOMS = df_VAERSSYMPTOMS.reset_index().drop("index",axis=1)

In [7]:
df_VAERSSYMPTOMS

,VAERS_ID,SYMPTOM1,SYMPTOMVERSION1,SYMPTOM2,SYMPTOMVERSION2,SYMPTOM3,SYMPTOMVERSION3,SYMPTOM4,SYMPTOMVERSION4,SYMPTOM5,SYMPTOMVERSION5
0,25001,Agitation,8.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,25003,Delirium,8.1,Hypokinesia,8.1,Hypotonia,8.1,NaN,NaN,NaN,NaN
2,25004,Chills,8.1,Dermatitis contact,8.1,Oedema genital,8.1,Pelvic pain,8.1,NaN,NaN
3,25005,Arthritis,8.1,Injection site oedema,8.1,Injection site reaction,8.1,NaN,NaN,NaN,NaN
4,25006,Convulsion,8.1,Dizziness,8.1,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2362321,2787677,Muscle spasms,27.0,Musculoskeletal stiffness,27.0,Pain,27.0,Pain in extremity,27.0,Periarthritis,27.0
2362322,2787677,Physical disability,27.0,Product administered at inappropriate site,27.0,Shoulder injury related to vaccine administration,27.0,Sleep disorder,27.0,Tendonitis,27.0
2362323,2787677,Wrong technique in product usage process,27.0,X-ray limb abnormal,27.0,NaN,NaN,NaN,NaN,NaN,NaN
2362324,2787678,Tinnitus,27.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
for i in range(len(VAERSVAX_CSVS)):
    if i==0:
        df_VAERSVAX = pd.read_csv(os.path.join(MAIN_DIRS,VAERSVAX_CSVS[i]),encoding='ISO-8859-1')
    else:
        df_i = pd.read_csv(os.path.join(MAIN_DIRS,VAERSVAX_CSVS[i]),encoding='ISO-8859-1')
        df_VAERSVAX = pd.concat([df_VAERSVAX,df_i],axis=0)
        df_VAERSVAX = df_VAERSVAX.reset_index().drop("index",axis=1)

In [9]:
df_VAERSVAX

,VAERS_ID,VAX_TYPE,VAX_MANU,VAX_LOT,VAX_DOSE_SERIES,VAX_ROUTE,VAX_SITE,VAX_NAME
0,25001,DTP,CONNAUGHT LABORATORIES,9Q01042,UNK,IM,NaN,DTP (NO BRAND NAME)
1,25003,DTP,LEDERLE LABORATORIES,259962,4,IM,NaN,DTP (TRI-IMMUNOL)
2,25003,OPV,PFIZER\WYETH,241950,4,PO,MO,"POLIO VIRUS, ORAL (ORIMUNE)"
3,25004,OPV,PFIZER\WYETH,232961,UNK,NaN,NaN,"POLIO VIRUS, ORAL (ORIMUNE)"
4,25005,TD,LEDERLE LABORATORIES,247955,UNK,IM,NaN,TD ADSORBED (NO BRAND NAME)
...,...,...,...,...,...,...,...,...
2333262,2787678,MENB,NOVARTIS VACCINES AND DIAGNOSTICS,G334J,1,NaN,RA,MENINGOCOCCAL B (BEXSERO)
2333263,2787678,MNQ,SANOFI PASTEUR,U8284AB,UNK,NaN,LA,MENINGOCOCCAL CONJUGATE (MENQUADFI)
2333264,2787679,DTAPIPV,SANOFI PASTEUR,U8009AB,UNK,IM,LL,DTAP + IPV (QUADRACEL)
2333265,2787679,MMR,MERCK & CO. INC.,X019107,UNK,SC,RL,MEASLES + MUMPS + RUBELLA (MMR II)


The **VAERS_ID** can be the key to merge all CSV's together.  

In [52]:
df_COMBINED = pd.merge(df_VAERSDATA,df_VAERSSYMPTOMS,on=["VAERS_ID"])
df_COMBINED = pd.merge(df_COMBINED,df_VAERSVAX,on=["VAERS_ID"])
df_COMBINED.head()

,VAERS_ID,RECVDATE,STATE,AGE_YRS,CAGE_YR,CAGE_MO,SEX,RPT_DATE,SYMPTOM_TEXT,DIED,...,SYMPTOMVERSION4,SYMPTOM5,SYMPTOMVERSION5,VAX_TYPE,VAX_MANU,VAX_LOT,VAX_DOSE_SERIES,VAX_ROUTE,VAX_SITE,VAX_NAME
0,25001,07/02/1990,WI,0.2,NaN,NaN,F,NaN,Loud intense cry with screaming for 1 1/2 hrs....,NaN,...,NaN,NaN,NaN,DTP,CONNAUGHT LABORATORIES,9Q01042,UNK,IM,NaN,DTP (NO BRAND NAME)
1,25003,07/02/1990,TX,0.8,NaN,NaN,M,NaN,"Hypotonic, Hyporesponsive episode, Infant died...",Y,...,NaN,NaN,NaN,DTP,LEDERLE LABORATORIES,259962,4,IM,NaN,DTP (TRI-IMMUNOL)
2,25003,07/02/1990,TX,0.8,NaN,NaN,M,NaN,"Hypotonic, Hyporesponsive episode, Infant died...",Y,...,NaN,NaN,NaN,OPV,PFIZER\WYETH,241950,4,PO,MO,"POLIO VIRUS, ORAL (ORIMUNE)"
3,25004,07/02/1990,NY,0.9,NaN,NaN,M,NaN,"Pt developed chills for approx. 1 hr, felt ach...",NaN,...,8.1,NaN,NaN,OPV,PFIZER\WYETH,232961,UNK,NaN,NaN,"POLIO VIRUS, ORAL (ORIMUNE)"
4,25005,07/02/1990,OK,NaN,NaN,NaN,U,NaN,7 patients within 2 weeks have reported joint ...,NaN,...,NaN,NaN,NaN,TD,LEDERLE LABORATORIES,247955,UNK,IM,NaN,TD ADSORBED (NO BRAND NAME)


## 1.1 Data Cleaning

We want to see what columns are in the combined dataset.

In [53]:
print(sorted(list(df_COMBINED.columns)))

['AGE_YRS', 'ALLERGIES', 'BIRTH_DEFECT', 'CAGE_MO', 'CAGE_YR', 'CUR_ILL', 'DATEDIED', 'DIED', 'DISABLE', 'ER_ED_VISIT', 'ER_VISIT', 'FORM_VERS', 'HISTORY', 'HOSPDAYS', 'HOSPITAL', 'LAB_DATA', 'L_THREAT', 'NUMDAYS', 'OFC_VISIT', 'ONSET_DATE', 'OTHER_MEDS', 'PRIOR_VAX', 'RECOVD', 'RECVDATE', 'RPT_DATE', 'SEX', 'SPLTTYPE', 'STATE', 'SYMPTOM1', 'SYMPTOM2', 'SYMPTOM3', 'SYMPTOM4', 'SYMPTOM5', 'SYMPTOMVERSION1', 'SYMPTOMVERSION2', 'SYMPTOMVERSION3', 'SYMPTOMVERSION4', 'SYMPTOMVERSION5', 'SYMPTOM_TEXT', 'TODAYS_DATE', 'VAERS_ID', 'VAX_DATE', 'VAX_DOSE_SERIES', 'VAX_LOT', 'VAX_MANU', 'VAX_NAME', 'VAX_ROUTE', 'VAX_SITE', 'VAX_TYPE', 'V_ADMINBY', 'V_FUNDBY', 'X_STAY']


There is several duplicates in this dataset that need to be addressed.

In [54]:
df_COMBINED[df_COMBINED.duplicated(["SYMPTOM_TEXT"])].sort_values(by="SYMPTOM_TEXT").head(10)

,VAERS_ID,RECVDATE,STATE,AGE_YRS,CAGE_YR,CAGE_MO,SEX,RPT_DATE,SYMPTOM_TEXT,DIED,...,SYMPTOMVERSION4,SYMPTOM5,SYMPTOMVERSION5,VAX_TYPE,VAX_MANU,VAX_LOT,VAX_DOSE_SERIES,VAX_ROUTE,VAX_SITE,VAX_NAME
48620,52276,05/03/1993,GA,5.0,5.0,NaN,M,02/11/1993,! 11PM started running fever; acted restless; ...,NaN,...,8.1,Pyrexia,8.1,MMR,MERCK & CO. INC.,0557V,2,SC,RA,MEASLES + MUMPS + RUBELLA (MMR II)
48624,52276,05/03/1993,GA,5.0,5.0,NaN,M,02/11/1993,! 11PM started running fever; acted restless; ...,NaN,...,NaN,NaN,NaN,OPV,PFIZER\WYETH,0657M,4,PO,MO,"POLIO VIRUS, ORAL (ORIMUNE)"
48623,52276,05/03/1993,GA,5.0,5.0,NaN,M,02/11/1993,! 11PM started running fever; acted restless; ...,NaN,...,NaN,NaN,NaN,MMR,MERCK & CO. INC.,0557V,2,SC,RA,MEASLES + MUMPS + RUBELLA (MMR II)
48622,52276,05/03/1993,GA,5.0,5.0,NaN,M,02/11/1993,! 11PM started running fever; acted restless; ...,NaN,...,NaN,NaN,NaN,DTP,CONNAUGHT LABORATORIES,2M31091,5,IM,LA,DTP (NO BRAND NAME)
48621,52276,05/03/1993,GA,5.0,5.0,NaN,M,02/11/1993,! 11PM started running fever; acted restless; ...,NaN,...,8.1,Pyrexia,8.1,OPV,PFIZER\WYETH,0657M,4,PO,MO,"POLIO VIRUS, ORAL (ORIMUNE)"
1649534,1212092,04/15/2021,OH,61.0,61.0,NaN,F,NaN,! day after the second dose patient developed ...,NaN,...,24.0,Blood immunoglobulin G,24.0,COVID19,PFIZER\BIONTECH,NaN,2,NaN,NaN,COVID19 (COVID19 (PFIZER-BIONTECH))
1649535,1212092,04/15/2021,OH,61.0,61.0,NaN,F,NaN,! day after the second dose patient developed ...,NaN,...,24.0,Blood urea normal,24.0,COVID19,PFIZER\BIONTECH,NaN,2,NaN,NaN,COVID19 (COVID19 (PFIZER-BIONTECH))
1649536,1212092,04/15/2021,OH,61.0,61.0,NaN,F,NaN,! day after the second dose patient developed ...,NaN,...,24.0,Oropharyngeal pain,24.0,COVID19,PFIZER\BIONTECH,NaN,2,NaN,NaN,COVID19 (COVID19 (PFIZER-BIONTECH))
1649537,1212092,04/15/2021,OH,61.0,61.0,NaN,F,NaN,! day after the second dose patient developed ...,NaN,...,24.0,Red blood cell sedimentation rate increased,24.0,COVID19,PFIZER\BIONTECH,NaN,2,NaN,NaN,COVID19 (COVID19 (PFIZER-BIONTECH))
1649538,1212092,04/15/2021,OH,61.0,61.0,NaN,F,NaN,! day after the second dose patient developed ...,NaN,...,24.0,NaN,NaN,COVID19,PFIZER\BIONTECH,NaN,2,NaN,NaN,COVID19 (COVID19 (PFIZER-BIONTECH))


In [55]:
print("Estimated number of duplicate rows = ",len(df_COMBINED[df_COMBINED.duplicated(["SYMPTOM_TEXT"])].sort_values(by="SYMPTOM_TEXT")))

Estimated number of duplicate rows =  1229749


Cleaning up the duplicates with **drop_duplicates**

In [56]:
df_COMBINED = df_COMBINED.drop_duplicates(["SYMPTOM_TEXT"])

We want to also visualize the number of days between receiving the vaccine (**VAX_DATE**) and when a person died (**DATEDIED**).  So we will make a new feature, **DAYS_to_DEATH**

In [57]:
df_COMBINED["DAYS_to_DEATH"] = pd.to_datetime(df_COMBINED.DATEDIED) - pd.to_datetime(df_COMBINED.VAX_DATE)
df_COMBINED["DAYS_to_DEATH"] = df_COMBINED["DAYS_to_DEATH"] /np.timedelta64(1,"D")

We can reset the index to cleanup our dropping of duplicates earlier.

In [58]:
df_COMBINED = df_COMBINED.reset_index().drop("index",axis=1)
df_COMBINED

,VAERS_ID,RECVDATE,STATE,AGE_YRS,CAGE_YR,CAGE_MO,SEX,RPT_DATE,SYMPTOM_TEXT,DIED,...,SYMPTOM5,SYMPTOMVERSION5,VAX_TYPE,VAX_MANU,VAX_LOT,VAX_DOSE_SERIES,VAX_ROUTE,VAX_SITE,VAX_NAME,DAYS_to_DEATH
0,25001,07/02/1990,WI,0.20,NaN,NaN,F,NaN,Loud intense cry with screaming for 1 1/2 hrs....,NaN,...,NaN,NaN,DTP,CONNAUGHT LABORATORIES,9Q01042,UNK,IM,NaN,DTP (NO BRAND NAME),NaN
1,25003,07/02/1990,TX,0.80,NaN,NaN,M,NaN,"Hypotonic, Hyporesponsive episode, Infant died...",Y,...,NaN,NaN,DTP,LEDERLE LABORATORIES,259962,4,IM,NaN,DTP (TRI-IMMUNOL),NaN
2,25004,07/02/1990,NY,0.90,NaN,NaN,M,NaN,"Pt developed chills for approx. 1 hr, felt ach...",NaN,...,NaN,NaN,OPV,PFIZER\WYETH,232961,UNK,NaN,NaN,"POLIO VIRUS, ORAL (ORIMUNE)",NaN
3,25005,07/02/1990,OK,NaN,NaN,NaN,U,NaN,7 patients within 2 weeks have reported joint ...,NaN,...,NaN,NaN,TD,LEDERLE LABORATORIES,247955,UNK,IM,NaN,TD ADSORBED (NO BRAND NAME),NaN
4,25006,07/02/1990,OH,16.00,16.0,NaN,F,NaN,16 yr old female feeling faint & then had seiz...,NaN,...,NaN,NaN,MMR,UNKNOWN MANUFACTURER,NaN,UNK,NaN,NaN,MEASLES + MUMPS + RUBELLA (NO BRAND NAME),NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1729649,2787665,08/30/2024,OK,4.00,NaN,NaN,U,NaN,there are no symptomatic adverse events report...,NaN,...,NaN,NaN,HPV9,MERCK & CO. INC.,W015894,UNK,NaN,NaN,HPV (GARDASIL 9),NaN
1729650,2787668,08/30/2024,IN,37.00,37.0,NaN,F,NaN,"Skin reactions, allergies, muscle weakness, bo...",NaN,...,Arthritis,27.0,COVID19,PFIZER\BIONTECH,Ew0153,1,IM,LA,COVID19 (COVID19 (PFIZER-BIONTECH)),NaN
1729651,2787674,08/30/2024,GA,0.25,0.0,0.3,U,NaN,No additional AE/No PQC.; HCP reported that RE...,NaN,...,NaN,NaN,DTAPHEPBIP,GLAXOSMITHKLINE BIOLOGICALS,NaN,UNK,NaN,NaN,DTAP + HEPB + IPV (PEDIARIX),NaN
1729652,2787675,08/30/2024,NY,NaN,NaN,NaN,F,NaN,Shingles outbreak after vaccination; This non-...,NaN,...,NaN,NaN,VARZOS,GLAXOSMITHKLINE BIOLOGICALS,UNK,UNK,NaN,NaN,ZOSTER (SHINGRIX),NaN


We can see what all the possible vaccines are with the unique **VAX_TYPE**

In [59]:
print(sorted(df_COMBINED.VAX_TYPE.unique()))

['6VAX-F', 'ADEN', 'ADEN_4_7', 'ANTH', 'BCG', 'CEE', 'CHIK', 'CHOL', 'COVID19', 'COVID19-2', 'DF', 'DPP', 'DT', 'DTAP', 'DTAPH', 'DTAPHEPBIP', 'DTAPIPV', 'DTAPIPVHIB', 'DTIPV', 'DTOX', 'DTP', 'DTPHEP', 'DTPHIB', 'DTPIHI', 'DTPIPV', 'DTPPHIB', 'DTPPVHBHPB', 'EBZR', 'FLU(H1N1)', 'FLU3', 'FLU4', 'FLUA3', 'FLUA4', 'FLUC3', 'FLUC4', 'FLUN(H1N1)', 'FLUN3', 'FLUN4', 'FLUR3', 'FLUR4', 'FLUX', 'FLUX(H1N1)', 'H5N1', 'HBHEPB', 'HBPV', 'HEP', 'HEPA', 'HEPAB', 'HEPATYP', 'HIBV', 'HPV2', 'HPV4', 'HPV9', 'HPVX', 'IPV', 'JEV', 'JEV1', 'JEVX', 'LYME', 'MEA', 'MEN', 'MENB', 'MENHIB', 'MER', 'MM', 'MMR', 'MMRV', 'MNP', 'MNQ', 'MNQHIB', 'MU', 'MUR', 'OPV', 'PER', 'PLAGUE', 'PNC', 'PNC10', 'PNC13', 'PNC15', 'PNC20', 'PPV', 'RAB', 'RSV', 'RUB', 'RV', 'RV1', 'RV5', 'RVX', 'SMALL', 'SMALLMNK', 'SSEV', 'TBE', 'TD', 'TDAP', 'TTOX', 'TYP', 'UNK', 'VARCEL', 'VARZOS', 'YF']


In order to focus primarily on COVID19, we can filter it by lowering the case and matching on the string **covid** under the **VAX_TYPE**

In [60]:
df_COVID = df_COMBINED[df_COMBINED.VAX_TYPE.str.lower().str.find("covid")!=-1]
df_COVID = df_COVID.reset_index().drop("index",axis=1)

We see that there is an data entry error for certain **VAX_LOT**, where there should only be 1 **VAX_MANU**.  So we will want to clean those up.  We will assume the most often occurance (MODE) can help clean up this error.

In [61]:
df_COVID['VAX_MANU'] = df_COVID.groupby('VAX_LOT')['VAX_MANU'].transform(lambda x: x.mode()[0])

In addition, we see that there is an data entry error for certain **VAX_LOT**, where there should only be 1 **VAX_TYPE**.  So we will want to clean those up.  We will assume the most often occurance (MODE) can help clean up this error.

In [62]:
df_COVID['VAX_TYPE'] = df_COVID.groupby('VAX_LOT')['VAX_TYPE'].transform(lambda x: x.mode()[0])

We will save off this for further analysis and ML model development.  

In [63]:
df_COVID.to_csv("df_COVID.csv",index=None)

## 1.2 Data Visualization

We will instantiate a copy of our df_COVID as df in order to visualize the dataset.

In [64]:
df = df_COVID.copy()

In order to visualize age & vax lot correlations for death.


We will create new variables:
1)  **Death_Count_per_Age_Group_per_VAX_LOT**, 
2) **Death_Percentage_per_Age_Group_per_VAX_LOT**, 
3) **Survive_Percentage_per_Age_Group_per_VAX_LOT**, 
4) **Survive_Count_per_Age_Group_per_VAX_LOT**
5) **Total_Count_per_Age_Group_per_VAX_LOT**

In [65]:
df['Age_Group'] = pd.cut(df['AGE_YRS'], bins=range(0, int(round(df['AGE_YRS'].max(),-1)+1), 10), right=False, labels=[f'{i}-{i+9}' for i in range(0, int(round(df['AGE_YRS'].max(),-1)), 10)])
total_counts = df.groupby(['Age_Group', 'VAX_LOT']).size().reset_index(name='Total_Count_per_Age_Group_per_VAX_LOT')
death_counts = df[df['DIED'] == 'Y'].groupby(['Age_Group', 'VAX_LOT']).size().reset_index(name='Death_Count_per_Age_Group_per_VAX_LOT')
survive_counts = df[df['DIED'] != 'Y'].groupby(['Age_Group', 'VAX_LOT']).size().reset_index(name='Survive_Count_per_Age_Group_per_VAX_LOT')

df_merged = total_counts.merge(death_counts, on=['Age_Group', 'VAX_LOT'], how='left')
df_merged["Death_Count_per_Age_Group_per_VAX_LOT"] = df_merged['Death_Count_per_Age_Group_per_VAX_LOT'].fillna(0)
df_merged["Death_Percentage_per_Age_Group_per_VAX_LOT"] = (df_merged["Death_Count_per_Age_Group_per_VAX_LOT"] / df_merged["Total_Count_per_Age_Group_per_VAX_LOT"]) * 100
df = df.merge(df_merged[["Age_Group", "VAX_LOT", "Death_Percentage_per_Age_Group_per_VAX_LOT","Death_Count_per_Age_Group_per_VAX_LOT",'Total_Count_per_Age_Group_per_VAX_LOT']], on=['Age_Group', 'VAX_LOT'], how='left')
df['Death_Percentage_per_Age_Group_per_VAX_LOT'] = df['Death_Percentage_per_Age_Group_per_VAX_LOT'].fillna(0)

df_merged = total_counts.merge(survive_counts, on=['Age_Group', 'VAX_LOT'], how='left')
df_merged["Survive_Count_per_Age_Group_per_VAX_LOT"] = df_merged['Survive_Count_per_Age_Group_per_VAX_LOT'].fillna(0)
df_merged["Survive_Percentage_per_Age_Group_per_VAX_LOT"] = (df_merged["Survive_Count_per_Age_Group_per_VAX_LOT"] / df_merged["Total_Count_per_Age_Group_per_VAX_LOT"]) * 100
df = df.merge(df_merged[["Age_Group", "VAX_LOT", "Survive_Percentage_per_Age_Group_per_VAX_LOT","Survive_Count_per_Age_Group_per_VAX_LOT"]], on=['Age_Group', 'VAX_LOT'], how='left')
df['Survive_Percentage_per_Age_Group_per_VAX_LOT'] = df['Survive_Percentage_per_Age_Group_per_VAX_LOT'].fillna(0)

In order to visualize vax lot correlations for death.


We will create new variables:
1)  **Death_Count_per_VAX_LOT**, 
2) **Death_Percentage_per_VAX_LOT**, 
3) **Survive_Percentage_per_VAX_LOT**,
4) **Survive_Count_per_VAX_LOT**,
5) **Total_Count_per_VAX_LOT**

In [66]:
total_counts = df.groupby(['VAX_LOT']).size().reset_index(name='Total_Count_per_VAX_LOT')
death_counts = df[df['DIED'] == 'Y'].groupby(['VAX_LOT']).size().reset_index(name='Death_Count_per_VAX_LOT')
survive_counts = df[df['DIED'] != 'Y'].groupby(['VAX_LOT']).size().reset_index(name='Survive_Count_per_VAX_LOT')

df_merged = total_counts.merge(death_counts, on=['VAX_LOT'], how='left')
df_merged["Death_Count_per_VAX_LOT"] = df_merged['Death_Count_per_VAX_LOT'].fillna(0)
df_merged["Death_Percentage_per_VAX_LOT"] = (df_merged["Death_Count_per_VAX_LOT"] / df_merged["Total_Count_per_VAX_LOT"]) * 100
df = df.merge(df_merged[[ "VAX_LOT", "Death_Percentage_per_VAX_LOT","Death_Count_per_VAX_LOT",'Total_Count_per_VAX_LOT']], on=['VAX_LOT'], how='left')
df['Death_Percentage_per_VAX_LOT'] = df['Death_Percentage_per_VAX_LOT'].fillna(0)

df_merged = total_counts.merge(survive_counts, on=['VAX_LOT'], how='left')
df_merged["Survive_Count_per_VAX_LOT"] = df_merged['Survive_Count_per_VAX_LOT'].fillna(0)
df_merged["Survive_Percentage_per_VAX_LOT"] = (df_merged["Survive_Count_per_VAX_LOT"] / df_merged["Total_Count_per_VAX_LOT"]) * 100
df = df.merge(df_merged[[ "VAX_LOT", "Survive_Percentage_per_VAX_LOT","Survive_Count_per_VAX_LOT"]], on=['VAX_LOT'], how='left')
df['Survive_Percentage_per_VAX_LOT'] = df['Survive_Percentage_per_VAX_LOT'].fillna(0)

In order to visualize vax manufacturer correlations for death.


We will create new variables:
1)  **Death_Count_per_VAX_MANU**, 
2) **Death_Percentage_per_VAX_MANU**, 
3) **Survive_Percentage_per_VAX_MANU**, 
4) **Survive_Count_per_VAX_MANU**,
5) **Total_Count_per_VAX_MANU**

In [67]:
total_counts = df.groupby(['VAX_MANU']).size().reset_index(name='Total_Count_per_VAX_MANU')
death_counts = df[df['DIED'] == 'Y'].groupby(['VAX_MANU']).size().reset_index(name='Death_Count_per_VAX_MANU')
survive_counts = df[df['DIED'] != 'Y'].groupby(['VAX_MANU']).size().reset_index(name='Survive_Count_per_VAX_MANU')

df_merged = total_counts.merge(death_counts, on=['VAX_MANU'], how='left')
df_merged["Death_Count_per_VAX_MANU"] = df_merged['Death_Count_per_VAX_MANU'].fillna(0)
df_merged["Death_Percentage_per_VAX_MANU"] = (df_merged["Death_Count_per_VAX_MANU"] / df_merged["Total_Count_per_VAX_MANU"]) * 100
df = df.merge(df_merged[[ "VAX_MANU", "Death_Percentage_per_VAX_MANU","Death_Count_per_VAX_MANU",'Total_Count_per_VAX_MANU']], on=['VAX_MANU'], how='left')
df['Death_Percentage_per_VAX_MANU'] = df['Death_Percentage_per_VAX_MANU'].fillna(0)

df_merged = total_counts.merge(survive_counts, on=['VAX_MANU'], how='left')
df_merged["Survive_Count_per_VAX_MANU"] = df_merged['Survive_Count_per_VAX_MANU'].fillna(0)
df_merged["Survive_Percentage_per_VAX_MANU"] = (df_merged["Survive_Count_per_VAX_MANU"] / df_merged["Total_Count_per_VAX_MANU"]) * 100
df = df.merge(df_merged[[ "VAX_MANU", "Survive_Percentage_per_VAX_MANU","Survive_Count_per_VAX_MANU"]], on=['VAX_MANU'], how='left')
df['Survive_Percentage_per_VAX_MANU'] = df['Survive_Percentage_per_VAX_MANU'].fillna(0)

Altair limits how many rows can be in a dataframe for their visualization by default.  So we will disable the max row limit for our analysis.

In [68]:
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

We might want to see what a minimum sample size would be needed for knowing if these correlated features have any statistical significance.

In [69]:
effect_size = 0.2 
alpha = 0.05
power = 0.8 
power_analysis = GofChisquarePower()
sample_size = power_analysis.solve_power(effect_size=effect_size, alpha=alpha, power=power)
print(f"Minimum sample size needed: {int(sample_size)}")


Minimum sample size needed: 196


We can then filter our data by this *sample_size* from our **Total_Count_per_VAX_LOT** variable.

In [70]:
df = df[df["Total_Count_per_VAX_LOT"]>=sample_size].reset_index().drop(["index"],axis=1)

We can look at the **EXPECTED_RATE_per_VAX_LOT** to see what death rate we would expect on average for a given **VAX_LOT** by dividing the **Death_Count_per_VAX_MANU** byt he **Total_Count_per_VAX_MANU**

In [71]:
df["EXPECTED_RATE_per_VAX_LOT"] = 100*df["Death_Count_per_VAX_MANU"]/df["Total_Count_per_VAX_MANU"]
df

,VAERS_ID,RECVDATE,STATE,AGE_YRS,CAGE_YR,CAGE_MO,SEX,RPT_DATE,SYMPTOM_TEXT,DIED,...,Death_Count_per_VAX_LOT,Total_Count_per_VAX_LOT,Survive_Percentage_per_VAX_LOT,Survive_Count_per_VAX_LOT,Death_Percentage_per_VAX_MANU,Death_Count_per_VAX_MANU,Total_Count_per_VAX_MANU,Survive_Percentage_per_VAX_MANU,Survive_Count_per_VAX_MANU,EXPECTED_RATE_per_VAX_LOT
0,902418,12/15/2020,NJ,56.0,56.0,NaN,F,NaN,Patient experienced mild numbness traveling fr...,NaN,...,32.0,3284.0,99.025579,3252.0,2.011114,5671.0,281983.0,97.988886,276312.0,2.011114
1,902446,12/15/2020,WV,55.0,55.0,NaN,F,NaN,"felt warm, hot and face and ears were red and ...",NaN,...,32.0,3284.0,99.025579,3252.0,2.011114,5671.0,281983.0,97.988886,276312.0,2.011114
2,902464,12/15/2020,LA,42.0,42.0,NaN,M,NaN,within 15 minutes progressive light-headedness...,NaN,...,32.0,3284.0,99.025579,3252.0,2.011114,5671.0,281983.0,97.988886,276312.0,2.011114
3,902465,12/15/2020,AR,60.0,60.0,NaN,F,NaN,Pt felt wave come over body @ 1218 starting in...,NaN,...,32.0,3284.0,99.025579,3252.0,2.011114,5671.0,281983.0,97.988886,276312.0,2.011114
4,902468,12/15/2020,NaN,59.0,59.0,NaN,M,NaN,"Within 1 minute, patient complained of symptom...",NaN,...,32.0,3284.0,99.025579,3252.0,2.011114,5671.0,281983.0,97.988886,276312.0,2.011114
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
491730,2787546,08/29/2024,TX,74.0,74.0,NaN,M,NaN,"He got his vaccine, he noticed a few days late...",NaN,...,25.0,1751.0,98.572244,1726.0,2.011114,5671.0,281983.0,97.988886,276312.0,2.011114
491731,2787553,08/29/2024,FL,52.0,52.0,NaN,F,NaN,"Vaccine administration site never healed, beca...",NaN,...,9.0,1173.0,99.232737,1164.0,2.011114,5671.0,281983.0,97.988886,276312.0,2.011114
491732,2787604,08/29/2024,OH,NaN,NaN,NaN,U,NaN,5 Patients Received this Expired Dose: 13Jul20...,NaN,...,12.0,811.0,98.520345,799.0,2.011114,5671.0,281983.0,97.988886,276312.0,2.011114
491733,2787633,08/29/2024,CA,70.0,NaN,NaN,F,NaN,"extended sleepiness, 12 to 18 hours daily; Thi...",NaN,...,9.0,1173.0,99.232737,1164.0,2.011114,5671.0,281983.0,97.988886,276312.0,2.011114


We can now assign a **P-value** for each **VAX_LOT** using a binomial test.  If the p-value is less than 0.05, we can reject the null hypothesis and assume there might be some statistical significance to that **VAX_LOT**.

In [ ]:
df["P-value"] = 1
for unique_manu in df.VAX_MANU.unique():
    expected_rate_i = df[df.VAX_MANU==unique_manu]["EXPECTED_RATE_per_VAX_LOT"]
    expected_rate_i = expected_rate_i.unique()[0]/100
    # print(unique_manu)
    # print(expected_rate_i)
    for batch_i in df[(df.VAX_MANU==unique_manu)].VAX_LOT.unique():
        deaths = df[(df.VAX_MANU==unique_manu) & (df.VAX_LOT==batch_i)]["Death_Count_per_VAX_LOT"]
        deaths = deaths.unique()[0]
        total_participants = df[(df.VAX_MANU==unique_manu) & (df.VAX_LOT==batch_i)]["Total_Count_per_VAX_LOT"]
        total_participants = total_participants.unique()[0]
        # Perform binomial test
        p_value = binomtest(int(deaths), int(total_participants), float(expected_rate_i))
        index_i = list(df[(df.VAX_MANU==unique_manu) & (df.VAX_LOT==batch_i)]["P-value"].index)
        df["P-value"].loc[index_i]=p_value.pvalue
        # if p_value.pvalue<0.05:
        #     print("batch_i",batch_i)
        #     print(f"P-value: {p_value}")
        #     print("\n")

In [87]:
df

,VAERS_ID,RECVDATE,STATE,AGE_YRS,CAGE_YR,CAGE_MO,SEX,RPT_DATE,SYMPTOM_TEXT,DIED,...,Total_Count_per_VAX_LOT,Survive_Percentage_per_VAX_LOT,Survive_Count_per_VAX_LOT,Death_Percentage_per_VAX_MANU,Death_Count_per_VAX_MANU,Total_Count_per_VAX_MANU,Survive_Percentage_per_VAX_MANU,Survive_Count_per_VAX_MANU,EXPECTED_RATE_per_VAX_LOT,P-value
0,902418,12/15/2020,NJ,56.0,56.0,NaN,F,NaN,Patient experienced mild numbness traveling fr...,NaN,...,3284.0,99.025579,3252.0,2.011114,5671.0,281983.0,97.988886,276312.0,2.011114,0.000004
1,902446,12/15/2020,WV,55.0,55.0,NaN,F,NaN,"felt warm, hot and face and ears were red and ...",NaN,...,3284.0,99.025579,3252.0,2.011114,5671.0,281983.0,97.988886,276312.0,2.011114,0.000004
2,902464,12/15/2020,LA,42.0,42.0,NaN,M,NaN,within 15 minutes progressive light-headedness...,NaN,...,3284.0,99.025579,3252.0,2.011114,5671.0,281983.0,97.988886,276312.0,2.011114,0.000004
3,902465,12/15/2020,AR,60.0,60.0,NaN,F,NaN,Pt felt wave come over body @ 1218 starting in...,NaN,...,3284.0,99.025579,3252.0,2.011114,5671.0,281983.0,97.988886,276312.0,2.011114,0.000004
4,902468,12/15/2020,NaN,59.0,59.0,NaN,M,NaN,"Within 1 minute, patient complained of symptom...",NaN,...,3284.0,99.025579,3252.0,2.011114,5671.0,281983.0,97.988886,276312.0,2.011114,0.000004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
491730,2787546,08/29/2024,TX,74.0,74.0,NaN,M,NaN,"He got his vaccine, he noticed a few days late...",NaN,...,1751.0,98.572244,1726.0,2.011114,5671.0,281983.0,97.988886,276312.0,2.011114,0.088018
491731,2787553,08/29/2024,FL,52.0,52.0,NaN,F,NaN,"Vaccine administration site never healed, beca...",NaN,...,1173.0,99.232737,1164.0,2.011114,5671.0,281983.0,97.988886,276312.0,2.011114,0.001133
491732,2787604,08/29/2024,OH,NaN,NaN,NaN,U,NaN,5 Patients Received this Expired Dose: 13Jul20...,NaN,...,811.0,98.520345,799.0,2.011114,5671.0,281983.0,97.988886,276312.0,2.011114,0.378753
491733,2787633,08/29/2024,CA,70.0,NaN,NaN,F,NaN,"extended sleepiness, 12 to 18 hours daily; Thi...",NaN,...,1173.0,99.232737,1164.0,2.011114,5671.0,281983.0,97.988886,276312.0,2.011114,0.001133


In [96]:
invalid_lots = ["unknown","UNKNOWN","unk","Unknown"]
invalid_type = ["UNK"]

df_filtered = df.groupby('VAX_LOT').filter(lambda x: x['Death_Count_per_VAX_LOT'].min() > 10)
df_filtered = df_filtered[df_filtered.VAX_MANU.str.find("UNKNOWN")==-1]
df_filtered = df_filtered[df_filtered["P-value"]<0.05]
df_filtered = df_filtered.groupby("VAX_LOT").filter(lambda x: x["Death_Percentage_per_VAX_LOT"].min() > 1)
df_filtered = df_filtered[~df_filtered['VAX_LOT'].isin(invalid_lots)]
df_filtered = df_filtered[~df_filtered['VAX_TYPE'].isin(invalid_type)]
df_filtered["VAX_LOT"] = [w+": "+z for w, z in zip(df_filtered["VAX_MANU"], df_filtered["VAX_LOT"])]


df_filtered = df_filtered.drop_duplicates(["VAX_LOT", "Age_Group"])
df_filtered = df_filtered.reset_index().drop("index",axis=1)
selection = alt.selection_multi(fields=['VAX_LOT'], bind='legend')

histogram = alt.Chart(df_filtered).mark_bar().encode(
    x=alt.X("VAX_LOT", sort=alt.EncodingSortField(
        field="Death_Count_per_Age_Group_per_VAX_LOT", 
        order="descending"
    ), axis=alt.Axis(labelAngle=-45)),  
    y=alt.Y("sum(Death_Count_per_Age_Group_per_VAX_LOT):Q", title="Total Deaths (#)"),  
    color=alt.Color('VAX_MANU:N', scale=alt.Scale(scheme='category10')),  
    tooltip=[alt.Tooltip("VAX_LOT", title="Vaccine Lot"), 
        alt.Tooltip("VAX_TYPE", title="Vaccine TYPE"),
            alt.Tooltip("Death_Count_per_VAX_LOT", title="Total Deaths (Lot)"),
            alt.Tooltip("Total_Count_per_VAX_LOT", title="Total Count (Lot)"),
        alt.Tooltip("Death_Percentage_per_VAX_LOT", title="Death % (Lot)"),
                alt.Tooltip("P-value", title="P-value (Lot)"),]
).add_selection(
    selection
).properties(
    width=4000,
    title="Total Deaths by Vaccine Lot (Colored by Manufacturer) [P-value < 0.05]"
)

scatter = alt.Chart(df_filtered).mark_circle().encode(
    x=alt.X("VAX_LOT", sort=alt.EncodingSortField(
        field="Death_Count_per_VAX_LOT",  
        order="descending"
    ), axis=alt.Axis(labelAngle=-45)),  
    y=alt.Y("AGE_YRS",title="Age (Years)"),
    color=alt.Color('Age_Group:N', scale=alt.Scale(scheme='spectral')),  
    size="Death_Count_per_Age_Group_per_VAX_LOT",
    tooltip=[
        alt.Tooltip("Age_Group", title="Age Group"),
        alt.Tooltip("VAX_TYPE", title="Vaccine TYPE"),
        alt.Tooltip("VAX_LOT", title="Vaccine Lot"),
        alt.Tooltip("VAX_MANU", title="Vaccine Manufacturer"),  
        alt.Tooltip("Death_Count_per_VAX_LOT", title="Total Deaths (Lot)"),
        alt.Tooltip("Death_Percentage_per_VAX_LOT", title="Death % (Lot)"),
        alt.Tooltip("Death_Percentage_per_Age_Group_per_VAX_LOT", title="Death % (Age Group, Lot)"),
        alt.Tooltip("Death_Count_per_Age_Group_per_VAX_LOT", title="Death Count (Age Group, Lot)"),
            alt.Tooltip("Total_Count_per_VAX_LOT", title="Total Count (Lot)"),

    ]
).transform_filter(
    selection
).properties(
    width=4000,
    title="Deaths by Age Group per Selected Vaccine Lot (Colored by Age Group) [P-value < 0.05]"
)

final_chart = alt.vconcat(histogram, scatter).resolve_scale(
    color='independent' ,
    size='independent' 
)
# Create a new chart to visualize deaths by VAX_LOT with death percentage
vax_type_chart = alt.Chart(df_filtered).mark_bar().encode(
    x=alt.X("VAX_LOT", sort=alt.EncodingSortField(
        field="Death_Count_per_Age_Group_per_VAX_LOT", 
        order="descending"
    ), axis=alt.Axis(labelAngle=-45)),  
    y=alt.Y("mean(Death_Percentage_per_VAX_LOT):Q", title="Average Death Percentage (%)"),  
    color=alt.Color('VAX_TYPE:N', scale=alt.Scale(scheme='spectral')),  
    tooltip=[alt.Tooltip("VAX_LOT", title="Vaccine Lot"), 
             alt.Tooltip("VAX_TYPE", title="Vaccine TYPE"),
             alt.Tooltip("mean(Death_Percentage_per_VAX_LOT):Q", title="Avg. Death % (Lot)"),
             alt.Tooltip("Death_Count_per_VAX_LOT", title="Total Deaths (Lot)"),
             alt.Tooltip("Total_Count_per_VAX_LOT", title="Total Count (Lot)"),
                             alt.Tooltip("P-value", title="P-value (Lot)"),
    ]
).add_selection(
    selection
).properties(
    width=4000,
    title="Average Death Percentage by Vaccine Lot (Colored by Manufacturer) [P-value < 0.05]"
)

final_chart_with_vax_type = alt.vconcat(
    histogram, scatter, vax_type_chart
).resolve_scale(
    color='independent',
    size='independent'
)


final_chart_with_vax_type

alt.VConcatChart(...)

In [92]:
final_chart_with_vax_type.save("final_chart_with_vax_type.html")

We see above that there might be some very statistically siginficant vaccine lots (**VAX_LOT**) that correlated with patient's death (**DIED**).  We will further investigate how to create an ML model to predict if the patient died given the data.